# Evaluate CCLM soil moisture with ERA5 and ERA5-Land


Needed: conversion of absolute soil moisture to volumetric soil moisture

volumetric_so = W_SO (m water per layer) * bulk_density (soil g/cm³)

In [3]:
from pathlib import Path

import xarray as xr
import pandas as pd

import valenspy as vp #The Valenspy package
from valenspy.inputconverter_functions import ERA5_to_CF, CCLM_to_CF
from valenspy._utilities import load_xarray_from_data_sources

from yaml import safe_load

import numpy as np
import matplotlib.pyplot as plt


In [4]:

def geo_to_rot(coord, ds):
    ###
    # Converts a geographic (lon,lat) point to an (rlon,rlat) point
    # => coord: geographic lon - lat couple (degrees) => list or tuple
    # => ds: xarray dataset of a CCLM file
    ###
    # Read in COSMO-CLM data

    rp_lat = float(ds.rotated_pole.grid_north_pole_latitude)
    rp_lon = float(ds.rotated_pole.grid_north_pole_longitude)
    # Convert 
    co = np.deg2rad(coord)
    rp_lat = np.deg2rad(rp_lat); rp_lon = np.deg2rad(rp_lon)
    p_rlat = np.arcsin(np.sin(co[1])*np.sin(rp_lat) + np.cos(co[1])*np.cos(rp_lat)*np.cos(co[0]-rp_lon)) 
    p_rlon = np.arctan((np.cos(co[1])*np.sin(co[0]-rp_lon)) / (np.cos(co[1])*np.sin(rp_lat)*np.cos(co[0]-rp_lon) - np.sin(co[1])*np.cos(rp_lat))) 
    p_rlat = np.rad2deg(p_rlat); p_rlon = np.rad2deg(p_rlon)
    # Return rlon-rlat couple
    return [p_rlon,p_rlat]

In [5]:
machine = "hortense"
variable = "tsl"
ref_dataset = "ERA5"


experiments = ["EUR11_CO_TA_GC_TSO",
               "EUR11_CO_TT_GC_TSO", 
               "EUR11_CO_TT_EC_TSO", 
               "EUR11_NU_TE_GC_TSO", 
               "EUR11_NU_TT_GC_TSO", 
               "EUR11_NU_TT_EC_TSO"]


# Define the coordinates
d_point_coords = {
    'opwijk'    : (4.18, 50.97), 
    'bertrix'    : (5.25, 49.854),
}
point_id = 'opwijk'
point_coord = d_point_coords[point_id]


months_to_analyse = [6,7,8]


In [6]:
# ERA5 files (manually loaded and converted)
obs_files = ['/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/hourly/soil_temperature_level_1/era5-hourly-europe-volumetric_soil_water_layer_11-1995.nc', 
'/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/hourly/soil_temperature_level_2/era5-hourly-europe-volumetric_soil_water_layer_2-1995.nc', 
'/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/hourly/soil_temperature_level_3/era5-hourly-europe-volumetric_soil_water_layer_3-1995.nc', 
'/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/hourly/soil_temperature_level_4/era5-hourly-europe-volumetric_soil_water_layer_4-1995.nc']





In [7]:
# 1. manually load ERA5

ds = load_xarray_from_data_sources(obs_files)

# rename dimensions if not yet renamed
if "lon" not in ds.coords:
    ds = ds.rename({"longitude": "lon"})
if "lat" not in ds.coords:
    ds = ds.rename({"latitude": "lat"})

# make sure lat and lon are sorted ascending
ds = ds.sortby('lat').sortby('lon')

# bugfix ERA5 (found in clh): replace valid_time by time
if "time" not in ds: 
    ds = ds.rename({'valid_time':'time'})

ds = ds.resample(time='1D').mean()  


# hardcoded from https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#ERA5:datadocumentation-Parameterlistings 

era5_soillayers = {'soil_depth': [0.07,0.28,0.1,2.89], 'thickness': [0.07,0.21,0.72,1.89], 
'layer': ['0 - 7 cm','7 - 28 cm', '28 - 100 cm ', '100 - 289 cm']} #m 

da_obs= xr.concat([ds['stl1'], ds['stl2'], ds['stl3'], ds['stl4']], dim='soil_depth')

da_obs['soil_depth'] = era5_soillayers['soil_depth']

ds_obs = da_obs.to_dataset(name=variable)


FileNotFoundError: [Errno 2] No such file or directory: '/dodrio/scratch/projects/2022_200/project_input/External/observations/era5/europe/hourly/soil_temperature_level_1/era5-hourly-europe-volumetric_soil_water_layer_11-1995.nc'

In [1]:
def plot_point_soiltemperature(ref_dataset, obs_files, variable, experiments, point_coord, point_id,  months_to_analyse, ax=False): 

    # 1. manually load ERA5

    ds = load_xarray_from_data_sources(obs_files)

    # rename dimensions if not yet renamed
    if "lon" not in ds.coords:
        ds = ds.rename({"longitude": "lon"})
    if "lat" not in ds.coords:
        ds = ds.rename({"latitude": "lat"})

    # make sure lat and lon are sorted ascending
    ds = ds.sortby('lat').sortby('lon')

    # bugfix ERA5 (found in clh): replace valid_time by time
    if "time" not in ds: 
        ds = ds.rename({'valid_time':'time'})

    ds = ds.resample(time='1D').mean()  


    # hardcoded from https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#ERA5:datadocumentation-Parameterlistings 

    era5_soillayers = {'soil_depth': [0.07,0.28,0.1,2.89], 'thickness': [0.07,0.21,0.72,1.89], 
    'layer': ['0 - 7 cm','7 - 28 cm', '28 - 100 cm ', '100 - 289 cm']} #m 

    da_obs= xr.concat([ds['stl1'], ds['stl2'], ds['stl3'], ds['stl4']], dim='soil_depth')

    da_obs['soil_depth'] = era5_soillayers['soil_depth']

    ds_obs = da_obs.to_dataset(name=variable)

    # select point from obs

    ds_obs_point = ds_obs.sel(lon=point_coord[0],lat=point_coord[1], method='nearest')
    da_obs_point = ds_obs_point[variable].sel(time=ds_obs['time'].dt.month.isin(months_to_analyse))
    da_obs_point_timmean = da_obs_point.mean('time')

    # 2. Load CCLM experiments

    src_path = Path("../src/valenspy")

    # define postprocessing base directory of CCLM
    postproc_base_dir = "/dodrio/scratch/projects/2022_200/RCS/CORDEXBE2/postprocessing/"

    # get experiment name

    # get the file directory based on the model name. 

    # reference dataset to compare to
    model_name = 'CCLM'

    # get lookup file for model
    with open(src_path / "ancilliary_data" / Path(model_name+"_lookup.yml")) as file:
        mod_LOOKUP = safe_load(file)



    # get CCLM variable corresponding to the requested variable using its look-up table
    mod_var = mod_LOOKUP[variable]['raw_name']


    d_da_mod_point = {}
    for experiment in experiments: 

        # define the path
        directory = Path(postproc_base_dir + experiment +'/'+mod_var + '/')

        # define the CCLM files for the corresponding variable
        cclm_files = list(directory.glob(mod_var+"_daymean.nc")) # Select all the netCDF files in the directory

        #Import Converter - This input converter will not do anything to the data.
        ic = vp.InputConverter(CCLM_to_CF)
        ds_mod = ic.convert_input(cclm_files) # Convert the input to the correct format

        ds_mod = ds_mod.rename({'soil1':'soil_depth'})

        coord_points_rotated = geo_to_rot(point_coord, ds_mod)
        ds_mod_point = ds_mod.sel(rlon = coord_points_rotated[0], rlat = coord_points_rotated[1], method='nearest')

        da_mod_point = ds_mod_point[variable].sel(time=ds_mod['time'].dt.month.isin(months_to_analyse))

        da_mod_point_timmean = da_mod_point.mean('time')

        d_da_mod_point[experiment] = da_mod_point_timmean



    if not ax: 
        fig, ax = plt.subplots(figsize = (7,3))

    ax.plot(da_obs_point_timmean.values, da_obs_point_timmean['soil_depth'].values, color='k', label = ref_dataset)
    for experiment in experiments: 
        ax.plot(d_da_mod_point[experiment].values, d_da_mod_point[experiment]['soil_depth'].values,  label=experiment)

    ax.invert_yaxis()
    ax.set_title(' ', loc='center')
    ax.set_title(f"{point_id} ({point_coord[1]}°N,  {point_coord[0]}°E)", loc='right');
    #ax.set_title(f"Mean soil temperature in {months_to_analyse}", loc='left')
    ax.set_ylabel('Soil depth (m)')
    ax.set_ylim([12,-0.5])
    ax.axhline(y=0, color="gray", linewidth = 0.5)

    return ax



In [2]:
def plot_timeseries_fourpoints(variable, ref_dataset, obs_files, experiments, d_point_coords,  months_to_analyse):

    "Wrapper function to combine 4 point timeseries plot into one figure, with nice layout (legend and title)"
    fig, axes = plt.subplots(1,2,figsize = (6,5))
    axes = axes.flatten()

    for i,point_id in enumerate(d_point_coords): 
        ax = axes[i]
        ax = plot_point_soiltemperature(ref_dataset, obs_files, variable, experiments, point_coord, point_id,  months_to_analyse, ax=ax)
        if i ==1: 
            ax.legend(frameon=False, loc='lower right')

    fig.tight_layout()

In [3]:
plot_timeseries_fourpoints(variable, ref_dataset, obs_files, experiments, d_point_coords,  months_to_analyse)

NameError: name 'variable' is not defined